# GCS Driveline Sorting

## 1. Load Data:

In [26]:
# Load in libraries
import pandas as pd

# CHANGE EACH YEAR
FILENAME = '2021-2022 Cameron data.xls'

# Load in Excel file and rename columns:
xl = pd.ExcelFile(FILENAME)
df = xl.parse(sheet_name=0, names=['student', 'grade', 'parent'])
df = df.sort_values(['student', 'parent'])
df.head()

,student,grade,parent
0,"Abbott, Mercy",7,"Abbott, Joshua and Michelle"
1,"Akers, Nash",5,"Akers, Trent and Necole"
2,"Allen, Brody",6,"Allen, Joshua and Carissa"
3,"Allen, Neveya",4,"Allen, Joshua and Carissa"
4,"Allison, Bennett",K5,"Allison, Brian and Lily"


## 2. Clean and Format Data:

We need to do a few things before we can assign release groups:
1. Remove duplicate students. 
2. We need to convert 'PK4' and 'K5' to number that we can sort on. We'll make K5 equal to 0
and we'll make PK4 -1. We'll convert them back at the end.
3. Convert grade to an integer. Right now they are all stored as string objects.

In [27]:
# Drop duplicates:
starting_len = len(df)
df.drop_duplicates('student', inplace=True)
ending_len = len(df)
print(f'Dropped {starting_len - ending_len} duplicate students.')

# Drop rows with empty grades
df.dropna(subset=['grade'], inplace=True)
no_grade_len = len(df)
print(f'Dropped {ending_len - no_grade_len} student(s) with missing grades.')

# Assign a new field called 'grade_new' that converts K5 and PK4 to 0 and -1 respectivly
df['grade_new'] = df['grade'].apply(lambda x: -1 if x=='PK4' else (0 if x=='K5' else int(x)))
df.head(10)

Dropped 148 duplicate students.
Dropped 1 student(s) with missing grades.


,student,grade,parent,grade_new
0,"Abbott, Mercy",7,"Abbott, Joshua and Michelle",7
1,"Akers, Nash",5,"Akers, Trent and Necole",5
2,"Allen, Brody",6,"Allen, Joshua and Carissa",6
3,"Allen, Neveya",4,"Allen, Joshua and Carissa",4
4,"Allison, Bennett",K5,"Allison, Brian and Lily",0
5,"Allison, Mila",2,"Allison, Brian and Lily",2
6,"Ames, Isabella",8,"Ames, Tom and Richele",8
7,"Anderson, Logan",8,"Anderson, Jason and Kelli",8
8,"Anderson, Savannah",8,"Anderson, Scot and Holly",8
9,"Arreola, Elias",K5,"Arreola, Elias and Arlene",0


In [28]:
# Assign each student the 'max' of the families grade:
df['release_grade'] = df.groupby('parent')['grade_new'].transform('max')
df.head(10)

,student,grade,parent,grade_new,release_grade
0,"Abbott, Mercy",7,"Abbott, Joshua and Michelle",7,7
1,"Akers, Nash",5,"Akers, Trent and Necole",5,5
2,"Allen, Brody",6,"Allen, Joshua and Carissa",6,6
3,"Allen, Neveya",4,"Allen, Joshua and Carissa",4,6
4,"Allison, Bennett",K5,"Allison, Brian and Lily",0,2
5,"Allison, Mila",2,"Allison, Brian and Lily",2,2
6,"Ames, Isabella",8,"Ames, Tom and Richele",8,8
7,"Anderson, Logan",8,"Anderson, Jason and Kelli",8,8
8,"Anderson, Savannah",8,"Anderson, Scot and Holly",8,8
9,"Arreola, Elias",K5,"Arreola, Elias and Arlene",0,0


In [29]:
# Convert 0 and -1 back to K5 and PK4
df['release_grade'] = df['release_grade'].apply(lambda x: 'PK4' if x==-1 else ('K5' if x==0 else x))
# Drop the grade_new field we created. We won't need it anymore. 
df.drop(columns='grade_new', inplace=True)

In [31]:
# Create the 'has_sibling' field.
family_count_dict = df.groupby('parent').count()['student'].to_dict() 
df['has_sibling'] = [1 if family_count_dict[x] > 1 else 0 for x in df['parent']]

Now that we have the grade and the sibling flag let's create some example groupings. We'll make five groups:
1. PK4
2. K-2
3. 3-5
4. 6-8 No Siblings 
5. 6-8 with Siblings

In [33]:
def release_group(row):
    if row['release_grade'] in ['K5', 1, 2]:
        return 'K-2'
    elif row['release_grade'] in [3, 4, 5]:
        return '3-5'
    elif (row['release_grade'] in [6, 7, 8]) & (row['has_sibling']==1):
        return '6-8 Siblings'
    elif (row['release_grade'] in [6, 7, 8]) & (row['has_sibling']!=1):
        return '6-8 No Siblings' 
    else:
        return 'PK4'

df['release_group'] = df.apply(release_group, axis=1)

In [34]:
# Count by release group
df_group = df.groupby('release_group').agg({'student':'count', 'parent':'nunique'}) 

# Count by release grade
df_grade = df.groupby('release_grade').agg({'student':'count', 'parent':'nunique'})

# Write to Excel
SAVE_FILE_PATH = 'driveline2021_results.xlsx'

writer = pd.ExcelWriter('{}.xlsx'.format(SAVE_FILE_PATH), engine='xlsxwriter') 

# Write each dataframe to a different worksheet.
df.to_excel(writer, sheet_name='Raw Data')
df_group.to_excel(writer, sheet_name='Possible Group Summary') 
df_grade.to_excel(writer, sheet_name='Release Grade Summary')

# Close the Pandas Excel writer and output the Excel file.
writer.save()